In [3]:
pip install deap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 2.7 MB/s eta 0:00:00


In [10]:
# Importación de las bibliotecas necesarias
import random
import numpy as np
from deap import base, creator, tools, algorithms

# Definir la función de evaluación f(x) = x^(2 * x) + 1
def funcion_evaluacion(individuo):
    x = individuo[0]
    return (x ** (2 * x)) + 1,

# Configuración de DEAP
creator.create("MaximizarAptitud", base.Fitness, weights=(1.0,))  # Queremos maximizar la función
creator.create("Individuo", list, fitness=creator.MaximizarAptitud)

toolbox = base.Toolbox()
toolbox.register("atributo_int", random.randint, 1, 10)  # Generador de atributo para x entre 1 y 10

# Aumentar el número de atributos por individuo
toolbox.register("individuo", tools.initRepeat, creator.Individuo, toolbox.atributo_int, n=2)  # Dos atributos por individuo
toolbox.register("poblacion", tools.initRepeat, list, toolbox.individuo)

toolbox.register("evaluar", funcion_evaluacion)  # Función de evaluación

# Cambiar el cruce a cruce de un punto
toolbox.register("cruzar", tools.cxOnePoint)  # Cruce de un punto

# Mutación (cambio de bit con una probabilidad del 10%)
toolbox.register("mutar", tools.mutFlipBit, indpb=0.1)

# Selección (torneo con tamaño 3)
toolbox.register("seleccionar", tools.selTournament, tournsize=3)

# Parámetros del algoritmo genético
tamano_poblacion = 6
generaciones = 3
prob_cruce = 0.7
prob_mutacion = 0.2

# Ejecutar el algoritmo genético
def ejecutar_algoritmo_genetico():
    # Inicializar la población
    poblacion = toolbox.poblacion(n=tamano_poblacion)

    # Bucle a través de las generaciones
    for gen in range(generaciones):
        print(f"-- Generación {gen+1} --")

        # Seleccionar los individuos de la siguiente generación
        descendencia = toolbox.seleccionar(poblacion, len(poblacion))
        descendencia = list(map(toolbox.clone, descendencia))

        # Aplicar el cruce a los descendientes
        for hijo1, hijo2 in zip(descendencia[::2], descendencia[1::2]):
            if random.random() < prob_cruce:
                toolbox.cruzar(hijo1, hijo2)  # Cruce de un punto
                del hijo1.fitness.values
                del hijo2.fitness.values

        # Aplicar la mutación a los descendientes
        for mutante in descendencia:
            if random.random() < prob_mutacion:
                toolbox.mutar(mutante)
                del mutante.fitness.values

        # Evaluar la aptitud de los individuos con aptitud inválida
        individuos_invalidos = [ind for ind in descendencia if not ind.fitness.valid]
        aptitudes = map(toolbox.evaluar, individuos_invalidos)
        for ind, apt in zip(individuos_invalidos, aptitudes):
            ind.fitness.values = apt

        # Reemplazar la población antigua con los nuevos descendientes
        poblacion[:] = descendencia

        # Imprimir la población y su aptitud
        aptitudes = [ind.fitness.values[0] for ind in poblacion]
        print(f"  Población: {poblacion}")
        print(f"  Aptitud de la población: {aptitudes}")

ejecutar_algoritmo_genetico()



-- Generación 1 --
  Población: [[8, 8], [1, 4], [6, 6], [6, 6], [3, 6], [6, 4]]
  Aptitud de la población: [281474976710657.0, 2.0, 2176782337.0, 2176782337.0, 730.0, 2176782337.0]
-- Generación 2 --
  Población: [[8, 4], [6, 8], [8, 8], [8, 8], [6, 6], [6, 6]]
  Aptitud de la población: [281474976710657.0, 2176782337.0, 281474976710657.0, 281474976710657.0, 2176782337.0, 2176782337.0]
-- Generación 3 --
  Población: [[8, 4], [8, 8], [8, 8], [8, 8], [8, 8], [8, 8]]
  Aptitud de la población: [281474976710657.0, 281474976710657.0, 281474976710657.0, 281474976710657.0, 281474976710657.0, 281474976710657.0]


In [13]:
# Implementación sin DEAP (b)
import random
import numpy as np

# Definir la función de evaluación f(x) = x^(2x) + 1
def funcion_evaluacion(x):
    return (x ** (2 * x)) + 1

# Parámetros para el algoritmo genético
tamano_poblacion = 6
generaciones = 3
prob_cruce = 0.7
prob_mutacion = 0.2
valor_minimo = 1
valor_maximo = 10

# Inicializar la población aleatoriamente entre 1 y 10
def inicializar_poblacion(tamano, valor_min, valor_max):
    return [random.randint(valor_min, valor_max) for _ in range(tamano)]

# Función de selección usando selección por torneo
def seleccion_por_torneo(poblacion, puntajes, k=3):
    seleccionados = []
    for _ in range(len(poblacion)):
        aspirantes = random.sample(list(zip(poblacion, puntajes)), k)
        seleccionados.append(max(aspirantes, key=lambda x: x[1])[0])
    return seleccionados

# Función de cruce (cruce de un punto)
def cruce(padre1, padre2):
    if random.random() < prob_cruce:
        return (padre1 + padre2) // 2, (padre1 + padre2) // 2  # Devolver dos descendientes
    return padre1, padre2

# Función de mutación (cambia aleatoriamente el valor dentro del rango)
def mutacion(individuo, valor_min, valor_max):
    if random.random() < prob_mutacion:
        return random.randint(valor_min, valor_max)
    return individuo

# Ejecutar el algoritmo genético sin DEAP
def ejecutar_algoritmo_genetico_sin_deap():
    poblacion = inicializar_poblacion(tamano_poblacion, valor_minimo, valor_maximo)

    for gen in range(generaciones):
        print(f"-- Generación {gen+1} --")

        # Evaluar la puntuación de la población
        puntajes = [funcion_evaluacion(ind) for ind in poblacion]
        print(f"  Población: {poblacion}")
        print(f"  Puntuaciones: {puntajes}")

        # Selección
        seleccionados = seleccion_por_torneo(poblacion, puntajes)

        # Cruce y mutación
        siguiente_gen = []
        for i in range(0, len(seleccionados), 2):
            padre1, padre2 = seleccionados[i], seleccionados[i+1] if i+1 < len(seleccionados) else seleccionados[i]
            descendiente1, descendiente2 = cruce(padre1, padre2)
            siguiente_gen.append(mutacion(descendiente1, valor_minimo, valor_maximo))
            if len(siguiente_gen) < tamano_poblacion:
                siguiente_gen.append(mutacion(descendiente2, valor_minimo, valor_maximo))

        poblacion = siguiente_gen

ejecutar_algoritmo_genetico_sin_deap()



-- Generación 1 --
  Población: [2, 6, 10, 1, 9, 10]
  Puntuaciones: [17, 2176782337, 100000000000000000001, 2, 150094635296999122, 100000000000000000001]
-- Generación 2 --
  Población: [10, 10, 10, 5, 1, 10]
  Puntuaciones: [100000000000000000001, 100000000000000000001, 100000000000000000001, 9765626, 2, 100000000000000000001]
-- Generación 3 --
  Población: [3, 10, 9, 10, 7, 10]
  Puntuaciones: [730, 100000000000000000001, 150094635296999122, 100000000000000000001, 678223072850, 100000000000000000001]
